#### Data Pre-process

In [16]:
import pandas as pd
import os

#Load dataframe dari file csv
dataframe1 = pd.read_csv('Csv-data/tempat-penampungan-sampah-sementara-tps-di-wilayah-dki-jakarta-tahun-2019.csv')
data1 = dataframe1.loc[:,['wilayah','kecamatan']]
data2 = dataframe1.loc[:,['kecamatan','volume_sampah_rata_rata_per_hari(m3)']]
data2.to_csv('Csv-data/volume.csv',index=False)

In [5]:
#Pre-process data before filtering
for column_name, column_data in data1.items():
    data1[column_name] = data1[column_name].str.strip()
    data1[column_name] = data1[column_name].str.title()

reduced_data = data1.drop_duplicates()

In [7]:
#Sort data based on alphabetical order
sorted_data = reduced_data.sort_values(by=['wilayah','kecamatan'])
sorted_data.to_csv('Csv-data/sorted.csv', index=False)

#there are some typo that should be handled manually

In [10]:
#count data for amaount of tps
data_duplicate = data1['kecamatan'].value_counts()
data_count = pd.DataFrame({'kecamatan':data_duplicate.index, 'Jumlah':data_duplicate.values})

counted_data = data_count.sort_values(by=['kecamatan'])
counted_data.to_csv('Csv-data/counted.csv',index=False)

#there are some typo that should be handled manually

In [ ]:
#merge data from counting and sorting
final_sorted = pd.read_csv('Csv-data/sorted-final.csv')
final_counted = pd.read_csv('Csv-data/counted-final.csv')

merged = pd.merge(final_sorted,final_counted, on='kecamatan')

merged.to_csv('Csv-data/merged1.csv',index=False)

In [18]:
#before load data, correct some typo from the original csv file
volume = pd.read_csv('Csv-data/volume-final.csv')

#pre-process data before using it
volume['kecamatan'] = volume['kecamatan'].str.strip()
volume['kecamatan'] = volume['kecamatan'].str.title()
volume['volume_sampah_rata_rata_per_hari(m3)'] = volume['volume_sampah_rata_rata_per_hari(m3)'].astype(float)

#data processing
grouped_sum = volume.groupby('kecamatan')['volume_sampah_rata_rata_per_hari(m3)'].transform('sum')
volume['total_volume(m3)'] = grouped_sum

In [19]:
#select only necessary and delete duplicate data
total_volume = volume.loc[:,['kecamatan','total_volume(m3)']]
reduced_volume = total_volume.drop_duplicates()

reduced_volume.to_csv('Csv-data/reduced-volume.csv',index=False)
#there are some typo that should be handled manually

In [61]:
#Load dataframe from csv file
dataframe2 = pd.read_csv('Csv-data/data-jumlah-kepala-keluarga-berdasarkan-jenis-kelamin-per-kelurahan-tahun-2020.csv')

data3 = dataframe2.loc[:,['kabupaten_kota','kecamatan','jumlah']]

In [62]:
#data pre-processing
data3['jumlah'] = data3['jumlah'].astype(int)

data3_sum = data3.groupby('kecamatan')['jumlah'].transform('sum')
data3['jumlah_kk'] = data3_sum

In [ ]:
#select only necessary and delete duplicate data
jumlah_kk = data3.loc[:,['kabupaten_kota','kecamatan','jumlah_kk']]
reduced_kk = jumlah_kk.drop_duplicates()
reduced_kk['kabupaten_kota'] = reduced_kk['kabupaten_kota'].str.title()
reduced_kk['kecamatan'] = reduced_kk['kecamatan'].str.title()

reduced_kk.to_csv('Csv-data/reduced-kk.csv',index=False)

In [115]:
#Pre-process area rawan banjir
data_banjir = pd.read_csv('Csv-data/wilayah-rawan-banjir.csv')
data_banjir_sum = data_banjir.groupby('Kecamatan')['Frekuensi'].transform('sum')
data_banjir['frekuensi_total'] = data_banjir_sum

In [118]:
#reshape the data
data_banjir_2 = pd.DataFrame({
    'kecamatan':data_banjir['Kecamatan'],
    'frekuensi_total':data_banjir['frekuensi_total'],
})

data_banjir_2 = data_banjir_2.drop_duplicates()
data_banjir_2.to_csv('Csv-data/wilayah-rawan-banjir-2.csv',index=False)

In [119]:
#merge all data that we have gathered
data_merge = pd.read_csv('Csv-data/merged.csv')
data_kk = pd.read_csv('Csv-data/reduced-kk-final.csv')
data_volume = pd.read_csv('Csv-data/reduced-volume-final.csv')
data_banjir_3 = pd.read_csv('Csv-data/wilayah-rawan-banjir-2.csv')

temp_data_merge = data_merge.loc[:,['kecamatan','Jumlah']]
temp_data_kk = data_kk.loc[:,['kecamatan','jumlah_kk']]

temp_merged = pd.merge(temp_data_kk,temp_data_merge, on='kecamatan', how='outer')
temp_merged_2 = pd.merge(temp_merged,data_banjir_3,on='kecamatan',how='left')
final_merged = pd.merge(temp_merged_2,data_volume, on='kecamatan',how='left')

final_merged.to_csv('Csv-data/final_merged.csv',index=False)

#### Data Visualization : Histogram

In [170]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

Data1 = pd.read_csv('Csv-data/sorted-final.csv')
Data2 = pd.read_csv('Csv-data/final_merged.csv')

Final_data = pd.merge(Data2,Data1,on='kecamatan',how='left')

In [172]:
sum_column = ['jumlah_kk','Jumlah','frekuensi_total','total_volume(m3)']
Temp_data = pd.DataFrame({'wilayah':Final_data['wilayah']})
for column_name in sum_column:
    summing = Final_data.groupby('wilayah')[column_name].transform('sum')
    Temp_data[column_name] = summing

Final_Data = pd.DataFrame({
    'wilayah':Temp_data['wilayah'],
    'frekuensi':Temp_data['frekuensi_total'],
    'jumlah_kk':Temp_data['jumlah_kk'],
    'jumlah_tps':Temp_data['Jumlah'],
    'volume (m3)':Temp_data['total_volume(m3)'],
})

Final_Data = Final_Data.drop_duplicates()


In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(Final_Data[['frekuensi','jumlah_kk','jumlah_tps','volume (m3)']])
Final_Data[['frekuensi','jumlah_kk','jumlah_tps','volume (m3)']] = scaled_data

# Plot the scaled DataFrame
Final_Data.plot(x='wilayah', y=['frekuensi','jumlah_kk','jumlah_tps','volume (m3)'])
plt.title('Grafik')
plt.xlabel('wilayah')
plt.ylabel('Scaled Amount')
plt.savefig('images/Grafik-wilayah.png')
plt.show()

#### Data Visualization : Maps

In [121]:
import geopandas as gpd
import matplotlib.pyplot as plt

#Prepare the maps
gdf_jakbar = gpd.read_file('city-regency-simplified/id-jk-jakbar.min.geojson')
gdf_jakpus = gpd.read_file('city-regency-simplified/id-jk-jakpus.min.geojson')
gdf_jaksel = gpd.read_file('city-regency-simplified/id-jk-jaksel.min.geojson')
gdf_jaktim = gpd.read_file('city-regency-simplified/id-jk-jaktim.min.geojson')
gdf_jakut = gpd.read_file('city-regency-simplified/id-jk-jakut.min.geojson')

#combine the maps
gdf_jakarta = gpd.GeoDataFrame(pd.concat([gdf_jakbar,gdf_jakpus,gdf_jaksel,gdf_jaktim,gdf_jakut], ignore_index=True), crs=gdf_jakbar.crs)
gdf1 = gpd.GeoDataFrame({'kecamatan':gdf_jakarta['name'],'geometry':gdf_jakarta['geometry']})
gdf2 = pd.merge(gdf1,final_merged,on='kecamatan',how='left')

In [ ]:
from matplotlib.colors import Normalize

# Define the variable to use for coloring
variable = 'jumlah_kk'

# Define the colormap and normalization
cmap = 'jet'  # Example: Cool-Warm colormap
norm = Normalize(vmin=gdf2[variable].min(), vmax=gdf2[variable].max())

# Plot the GeoDataFrame with different colors based on the variable
gdf2.plot(column=variable, cmap=cmap, linewidth=0.8, edgecolor='black', norm=norm, legend=True)
plt.savefig('images/persebaran-jumlah-kk')

# Customize the plot
plt.title('Map for Jumlah-kk')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

plt.show()

In [ ]:
from matplotlib.colors import Normalize

# Define the variable to use for coloring
variable = 'Jumlah'

# Define the colormap and normalization
cmap = 'coolwarm'  # Example: Cool-Warm colormap
norm = Normalize(vmin=gdf2[variable].min(), vmax=gdf2[variable].max())

# Plot the GeoDataFrame with different colors based on the variable
gdf2.plot(column=variable, cmap=cmap, linewidth=0.8, edgecolor='black', norm=norm, legend=True)
plt.savefig('images/persebaran-jumlah-TPS.png')

# Customize the plot
plt.title('Map for Jumlah-TPS')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

plt.show()

In [ ]:
from matplotlib.colors import Normalize

# Define the variable to use for coloring
variable = 'total_volume(m3)'

# Define the colormap and normalization
cmap = 'plasma'  # Example: Cool-Warm colormap
norm = Normalize(vmin=gdf2[variable].min(), vmax=gdf2[variable].max())

# Plot the GeoDataFrame with different colors based on the variable
gdf2.plot(column=variable, cmap=cmap, linewidth=0.8, edgecolor='black', norm=norm, legend=True)
plt.savefig('images/persebaran-sampah-masuk-tps.png')

# Customize the plot
plt.title('Map for sampah masuk tps(m3)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

plt.show()

In [ ]:
from matplotlib.colors import Normalize

# Define the variable to use for coloring
variable = 'frekuensi_total'

# Define the colormap and normalization
cmap = 'jet'  # Example: Cool-Warm colormap
norm = Normalize(vmin=gdf2[variable].min(), vmax=gdf2[variable].max())

# Plot the GeoDataFrame with different colors based on the variable
gdf2.plot(column=variable, cmap=cmap, linewidth=0.8, edgecolor='black', norm=norm, legend=True)
plt.savefig('images/persebaran-Daerah-Rawan-Banjir.png')

# Customize the plot
plt.title('Map for Daerah Rawan Banjir')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

plt.show()

#### Corelation

In [ ]:
Data1 = pd.read_csv('Csv-data/sorted-final.csv')
Data2 = pd.read_csv('Csv-data/final_merged.csv')

Final_data = pd.merge(Data2,Data1,on='kecamatan',how='left')

drop_col = ['kecamatan','wilayah']
Corr_purpose = Final_data.drop(drop_col,axis=1)
Corr_purpose.columns = ['Jumlah_KK','Jumlah TPS', 'Frekuensi Banjir','Volume Sampah/Hari']
print(Corr_purpose)
Corr_purpose.corr()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


sns.heatmap(Corr_purpose.corr(), annot=True, cmap='YlGnBu', fmt=".2f")
plt.title('Tabel Korelasi')
plt.savefig('images/tabel-korelasi.png')
plt.show()